<a href="https://colab.research.google.com/github/SeyfGoumeida/Projet_Mixt_Coclus_Partie1/blob/main/Projet_Mixt_Coclus_Partie1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install nltk
!pip install gensim

In [17]:
import pandas as pd
import string
import re
import operator
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from spacy.lang.en.stop_words import STOP_WORDS
#------------------------------------------------ 
import os
from tensorflow.keras import layers, optimizers, losses, metrics, Model
from sklearn import preprocessing, model_selection
from IPython.display import display, HTML
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from spacy.lang.en.stop_words import STOP_WORDS
import string
import re
import operator
from functools import reduce
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from collections import Counter


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv('/content/classic4.csv',header=0,index_col=0)
df


,text,label
0,Computer-Aided Analysis and Design of Informat...,cacm
1,Milestones in Cataloging In the case of the pr...,cisi
2,childhood psychosis. a description is given of...,med
3,neonatal hepatitis or familial neonatal obstru...,med
4,Handbook of Comparative Librarianship The firs...,cisi
...,...,...
7090,throat and nose surgery in patients with const...,med
7091,Computers and Society: A Proposed Course for C...,cacm
7092,Romberg Integration (Algorithm 60),cacm
7093,Semantic Evaluation from Left to Right This pa...,cacm


In [3]:
df["label"].value_counts().to_frame()

,label
cacm,3204
cisi,1460
cran,1398
med,1033


In [16]:
from gensim.utils import tokenize
#sentance = [list(tokenize(s, deacc=True, lower=True)) for s in df['text']]
#Stop Words
#from gensim.parsing.preprocessing import remove_stopwords
#preprocceced_text = [remove_stopwords(s) for s in df['text']]
#Ponctuation


In [18]:

#punctuations
punctuations = string.punctuation
#stopwords
stop_words = list(STOP_WORDS)
def clean_text(
  string : str,
  ponctuation = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~',
  stop_words = stop_words)->str : 

  #URLs
  string = re.sub(r"https?://\S+|www\.\S+",'',string)
  #html 
  string = re.sub(r'<.*?>','',string)
  #Numbers
  string = re.sub(r'[0-9]+','',string) 

  #Remove punctuations
  for x in string.lower():
    if x in punctuations:
      string = string.replace(x,"")
  #To lower
  string = string.lower()
  #Remove stop words
  lemmatizer = WordNetLemmatizer()
  string = ' '.join([lemmatizer.lemmatize(word) for word in string.split() if word not in stop_words])
  #Cleaning the whitespaces
  string = re.sub(r'\s', ' ', string).strip()

  return string

In [21]:
abstract = df['text'].to_list()
abstracts_cleaned = [clean_text(x) for x in abstract]

In [22]:
abstracts_cleaned

['computeraided analysis design information system paper describes use computeraided analysis design development integrated financial management system navy material command support activity nmcsa computeraided analysis consists set procedure computer program specifically designed aid process application software design computer selection performance evaluation major component problem statement language problem statement analyzer generator alternative design performance evaluator statement requirement written ad accurately defined system analyzed problem statement analyzer ad ad problem definition supplemented additional information order create complete problem definition analyzed problem statement translated form necessary use soda system optimization design algorithm program generation alternative specification program module logical database structure',
 'milestone cataloging case present study drlehnus interested applying method nonstandard way method type literature addressed far

#Word2Vec

In [27]:
from gensim.utils import tokenize
sentance = [list(tokenize(s, deacc=True, lower=True)) for s in abstracts_cleaned]


In [29]:
from gensim.models import word2vec
model = word2vec.Word2Vec(sentance,min_count=2)
model.corpus_count

7095

In [71]:
vocab = model.wv.vocab
words = list(vocab)


In [38]:
print(abstracts_cleaned[0])

computeraided analysis design information system paper describes use computeraided analysis design development integrated financial management system navy material command support activity nmcsa computeraided analysis consists set procedure computer program specifically designed aid process application software design computer selection performance evaluation major component problem statement language problem statement analyzer generator alternative design performance evaluator statement requirement written ad accurately defined system analyzed problem statement analyzer ad ad problem definition supplemented additional information order create complete problem definition analyzed problem statement translated form necessary use soda system optimization design algorithm program generation alternative specification program module logical database structure


In [72]:
k = list(model.wv.vectors)
#Vecteur de mots 
print(k[0])
#la liste des mots 
words

[-1.4861796  -1.0390624  -1.3247006   0.35761395  1.4547045   0.394938
 -0.66569453 -0.4206104  -1.9452982  -0.7022081  -0.45507452 -0.2706417
  0.3522468   0.10104439  0.5832761   1.1771755  -1.2252976  -0.7890874
 -0.62955946 -0.40801987  0.14330554  0.16537447 -0.14941788 -0.12722085
  1.499833   -1.0184115   0.53670144  0.8530225  -1.5581743  -0.12619829
 -0.15089875  1.4267035   0.3973755   0.49693456  1.3108778   0.0279536
  0.17971206  0.5606212   0.17506042  0.3124802   0.6234704   0.89012873
 -1.539719    0.08997089 -0.6811319   0.19270973  1.2292511   0.92785156
 -0.6444904  -1.0625142   0.31550944  0.44282392  0.7789137  -0.69464207
  0.06901787  0.8881197   0.180809    0.98647743  0.87935454 -0.7799175
  0.254932    1.0841676   1.4308399  -1.731362    1.4939976   0.37829328
  0.34364772  0.07106487  1.3842719  -0.00855442 -0.21032196  1.3504659
 -0.0214967   0.00917266 -0.5906072  -0.15476596  0.29904252  0.19020352
 -0.13287422  0.50712204  0.7660098  -1.7873831   0.585941

['computeraided',
 'analysis',
 'design',
 'information',
 'system',
 'paper',
 'describes',
 'use',
 'development',
 'integrated',
 'financial',
 'management',
 'navy',
 'material',
 'command',
 'support',
 'activity',
 'consists',
 'set',
 'procedure',
 'computer',
 'program',
 'specifically',
 'designed',
 'aid',
 'process',
 'application',
 'software',
 'selection',
 'performance',
 'evaluation',
 'major',
 'component',
 'problem',
 'statement',
 'language',
 'analyzer',
 'generator',
 'alternative',
 'evaluator',
 'requirement',
 'written',
 'ad',
 'accurately',
 'defined',
 'analyzed',
 'definition',
 'supplemented',
 'additional',
 'order',
 'create',
 'complete',
 'translated',
 'form',
 'necessary',
 'optimization',
 'algorithm',
 'generation',
 'specification',
 'module',
 'logical',
 'database',
 'structure',
 'cataloging',
 'case',
 'present',
 'study',
 'interested',
 'applying',
 'method',
 'nonstandard',
 'way',
 'type',
 'literature',
 'addressed',
 'far',
 'proposed',
